# Reactant demo

<div class="alert alert-block alert-warning"> ⚠️ Currently only working for <b>synchronous clients</b>! </div>

In [18]:
using Revise
using Tenet
using EinExprs
using Reactant
using AbstractTrees
using Random

In [2]:
Random.seed!(0)
tn = rand(TensorNetwork, 15, 3; dim=(16,16))

TensorNetwork (#tensors=15, #inds=22)

In [3]:
path = einexpr(tn; optimizer=Exhaustive())

SizedEinExpr{Symbol}(EinExpr{Symbol}(Symbol[], EinExpr{Symbol}[EinExpr{Symbol}([:R], EinExpr{Symbol}[EinExpr{Symbol}([:K], EinExpr{Symbol}[]), EinExpr{Symbol}([:R, :K], EinExpr{Symbol}[])]), EinExpr{Symbol}([:R], EinExpr{Symbol}[EinExpr{Symbol}([:G, :N, :R], EinExpr{Symbol}[]), EinExpr{Symbol}([:G, :N], EinExpr{Symbol}[EinExpr{Symbol}([:H, :Q, :N], EinExpr{Symbol}[]), EinExpr{Symbol}([:H, :Q, :G], EinExpr{Symbol}[EinExpr{Symbol}([:C, :F, :Q], EinExpr{Symbol}[]), EinExpr{Symbol}([:H, :C, :F, :G], EinExpr{Symbol}[EinExpr{Symbol}([:D, :H, :O, :C], EinExpr{Symbol}[]), EinExpr{Symbol}([:D, :O, :F, :G], EinExpr{Symbol}[EinExpr{Symbol}([:B, :D, :F, :J], EinExpr{Symbol}[EinExpr{Symbol}([:B, :D, :F, :V], EinExpr{Symbol}[]), EinExpr{Symbol}([:V, :J], EinExpr{Symbol}[EinExpr{Symbol}([:P, :J], EinExpr{Symbol}[]), EinExpr{Symbol}([:V, :P], EinExpr{Symbol}[EinExpr{Symbol}([:E], EinExpr{Symbol}[]), EinExpr{Symbol}([:E, :V, :P], EinExpr{Symbol}[])])])]), EinExpr{Symbol}([:B, :O, :G, :J], EinExpr{Symbo

In [4]:
maximum(ndims, Branches(path))

4

In [5]:
maximum(length, Branches(path))

65536

In [6]:
log10(mapreduce(flops, +, Branches(path)))

7.940067535122738678444270654298377812457104636982098412933015342439109112833246

In [7]:
expr = Tenet.codegen(Val(:outplace), path.path)

quote
    #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:92 =#
    function var"##contract_compiled#225"(ssa1::Tensor, ssa2::Tensor, ssa3::Tensor, ssa4::Tensor, ssa5::Tensor, ssa6::Tensor, ssa7::Tensor, ssa8::Tensor, ssa9::Tensor, ssa10::Tensor, ssa11::Tensor, ssa12::Tensor, ssa13::Tensor, ssa14::Tensor, ssa15::Tensor)
        #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:92 =#
        #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:93 =#
        ssa16 = contract(ssa1, ssa2)
        ssa17 = contract(ssa9, ssa10)
        ssa18 = contract(ssa8, ssa17)
        ssa19 = contract(ssa7, ssa18)
        ssa20 = contract(ssa14, ssa15)
        ssa21 = contract(ssa13, ssa20)
        ssa22 = contract(ssa12, ssa21)
        ssa23 = contract(ssa11, ssa22)
        ssa24 = contract(ssa19, ssa23)
        ssa25 = contract(ssa6, ssa24)
        ssa26 = contract(ssa5, ssa25)
        ssa27 = contract(ssa4, ssa26)
        ssa28 = contract(ssa3, ssa27)
        ssa29 = contract(ssa16, ssa28)
        #= /home/

In [8]:
f = @eval $expr

##contract_compiled#225 (generic function with 2 methods)

In [9]:
ts = [tn[head(leaf)...] for leaf in Leaves(path)];

In [10]:
# warm up
f(ts...)

@time f(ts...)

  0.470928 seconds (2.79 k allocations: 9.386 MiB)


0-dimensional Tensor{Float64, 0, Array{Float64, 0}}:
5.991293430814615e21

In [11]:
ts′ = [Tensor(Reactant.ConcreteRArray(parent(tensor)), inds(tensor)) for tensor in ts]
g = Reactant.compile(f, Tuple(ts′))

#109 (generic function with 1 method)

In [12]:
# warm up
g(ts′...)

@time x = g(ts′...)

  0.003124 seconds (6 allocations: 208 bytes)


0-dimensional Tensor{Float64, 0, Reactant.ConcreteRArray{Float64, (), 0}}:
5.991293430814615e21

In [13]:
x

0-dimensional Tensor{Float64, 0, Reactant.ConcreteRArray{Float64, (), 0}}:
5.991293430814615e21